In [72]:
import cv2
import joblib
import os
from matplotlib import pyplot as plt
from src.utils import *
import pickle # alternative to joblib 
import json
from sklearn.metrics import classification_report
import random

In [54]:
# load the model
model_path = r"src\checkpoint\knn_custom_best.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

# load the parameters
params_path = r"src\checkpoint\knn_parameters.json"
with open(params_path, "r") as f:
    params = json.load(f)

IMAGE_SIZE = params["features_extractor__image_size"]
BIN_SIZE = params["features_extractor__bins"]
print(f"\nIMAGE SIZE: {IMAGE_SIZE} \nBIN SIZE: {BIN_SIZE}")


IMAGE SIZE: 512 
BIN SIZE: 16


gt:     [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1]
preds:  [1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1]
              precision    recall  f1-score   support

  notsmoking       0.86      0.86      0.86         7
     smoking       0.88      0.88      0.88         8

    accuracy                           0.87        15
   macro avg       0.87      0.87      0.87        15
weighted avg       0.87      0.87      0.87        15



In [76]:
test_path = "data/test/"
image_files = os.listdir(test_path)

label_names = ['notsmoking', 'smoking']
gt = []
preds = []
# number of wanted images
num_images = 15
if num_images > len(image_files):
    num_images = len(image_files)

# number of images per row
images_per_row = 5 if num_images > 5 else num_images//2
# calc number of rows
num_rows = (num_images // images_per_row) + (1 if num_images % images_per_row != 0 else 0)

# create a grid of images
fig, axes = plt.subplots(num_rows, images_per_row, figsize=(15, 5 * num_rows))

# shuffle images
random.seed(42) 
random.shuffle(image_files)
# iterate over all images
for i, image_name in enumerate(image_files[:num_images]):
    # create gt
    if image_name.startswith("smoking"):
        gt.append(1)
    elif image_name.startswith("notsmoking"):
        gt.append(0)
    
    
    # read image
    image_path = os.path.join(test_path, image_name)
    origin_image = cv2.imread(image_path)
    origin_image = cv2.cvtColor(origin_image, cv2.COLOR_BGR2RGB)  # Đổi màu từ BGR sang RGB

    # feature extraction and predict
    feature_image = cv2.resize(origin_image, (IMAGE_SIZE, IMAGE_SIZE))
    feature_image = cv2.cvtColor(feature_image, cv2.COLOR_RGB2HSV)
    feature = calc_hist(feature_image, BIN_SIZE)

    pred = model.predict([feature])[0]
    preds.append(pred)
    label = label_names[pred]



    # calculate row and col index
    row = i // images_per_row
    col = i % images_per_row

    # plot image
    axes[row, col].imshow(origin_image)
    axes[row, col].axis('off')  # Tắt hiển thị trục

    # add label
    axes[row, col].text(0.5, -0.01, label, ha='center', va='top', fontsize=16, color='red', transform=axes[row, col].transAxes)

# remove empty subplots
for i in range(num_images, num_rows * images_per_row):
    row = i // images_per_row
    col = i % images_per_row
    axes[row, col].axis('off')

plt.tight_layout()  # Điều chỉnh khoảng cách giữa các ảnh
plt.show()

print("gt:    ", gt)
print("preds: ", preds)
print(classification_report(gt, preds, target_names=label_names))


In [75]:

"""
Do không có ảnh test nên lấy tạm trong thư mục gốc để test
"""
import os
import shutil
import random

# Đường dẫn đến thư mục gốc và thư mục đích
source_folder = r"data\images"
destination_folder = r"data\test"
n = 15 # Số lượng ảnh mỗi loại cần sao chép

# Tạo thư mục test nếu chưa tồn tại
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Xóa toàn bộ file trong thư mục test
for file_name in os.listdir(destination_folder):
    file_path = os.path.join(destination_folder, file_name)
    if os.path.isfile(file_path):  # Kiểm tra file (bỏ qua thư mục con nếu có)
        os.remove(file_path)

# Liệt kê tất cả các file trong thư mục gốc
all_files = os.listdir(source_folder)

# Lọc file theo tên bắt đầu bằng "notsmoking" và "smoking"
notsmoking_files = [f for f in all_files if f.startswith("notsmoking")]
smoking_files = [f for f in all_files if f.startswith("smoking")]

# Lấy ngẫu nhiên n file từ mỗi danh sách
selected_notsmoking = random.sample(notsmoking_files, min(n, len(notsmoking_files)))
selected_smoking = random.sample(smoking_files, min(n, len(smoking_files)))

# Sao chép file sang thư mục test
for file_name in selected_notsmoking + selected_smoking:
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)
    shutil.copy(source_path, destination_path)

print(f"Đã sao chép {len(selected_notsmoking)} ảnh 'notsmoking' và {len(selected_smoking)} ảnh 'smoking' vào thư mục '{destination_folder}'.")


Đã sao chép 15 ảnh 'notsmoking' và 15 ảnh 'smoking' vào thư mục 'data\test'.
